# FEMA Region Geometry to GeoJson
This notebook will illustrate how to create GeoJson files for FEMA regions. One cell below will save all the regions in one file, while another will create one file for each. These files can be used as map layers within your tools/analysis and, will display natively in GitHub.

In [183]:
import json
import requests
import geojson

# define the api query
baseURL = "https://www.fema.gov/api/open/v2/FemaRegions?"

# call api 
response = requests.get(baseURL + '$metadata=off')
jsonData = response.json()

# lets display the metadata object that is sent with every response
#print(json.dumps(jsonData['FemaRegions'], indent=4, sort_keys=False))
print(str(len(jsonData['FemaRegions'])))

11


## Create GeoJson Features and Save

This will create individual files for each region with the properties/feature popup being the point representing the regional office. For the file containing all regions, the properties/feature popup will be anywhere on the region itself as some regions utilize multipolygons and non-contiguous shapes.


In [185]:
# initialize a feature collection list for all regions 
featureCollectAll = []

# iterate through regions and construct geojson collection for each
for region in jsonData['FemaRegions']:
    # initialize a feature collection list for this region
    featureCollect = [] 

    # headquarters does not have a region designation
    if region['region'] is None:
        region['region'] = 'NA'
        featId = 0
        filename = 'FEMA_Headquarters.geojson'
    else:
        featId = int(region['region'])
        filename = 'FEMA_Region_' + str(region['region']) + '.geojson'
    
    # build out a properties dictionary
    props = {
        'Region':region['region'],
        'Name':region['name'],
        'Address':region['address'],
        'City-St-Zip':(region['city'] + ", " + region['state'] + " " + region['zipCode']),
        'States':region['states'],
        'Last-Update':region['lastRefresh']
            }
    
    # build geojson geometry (HQ has no geometry)
    if region['regionGeometry'] is not None:
        if region['regionGeometry']['type'] == 'Polygon':
            geom = geojson.Polygon(region['regionGeometry']['coordinates'])
        else:
            geom = geojson.MultiPolygon(region['regionGeometry']['coordinates'])
    
        # create polygon type feature and append it to our collection
        featureCollect.append(Feature(geometry=geom, id=featId))
        # for the full file, we want the properties associated with the shape
        featureCollectAll.append(Feature(geometry=geom, properties=props, id=featId))
        
    # lets append a point that represents the office
    geom = geojson.Point(region['loc']['coordinates'])
    featureCollect.append(Feature(geometry=geom, properties=props))
    # don't want properties associated with points in full file
    featureCollectAll.append(Feature(geometry=geom))
    
    # create our geojson from our features collection
    geojsonFinal = geojson.FeatureCollection(featureCollect)

    # save as a file
    with open(filename, 'w') as fp:
       dump(geojsonFinal, fp)
    
# finally, lets save a file that contains all the regions
geojsonFinal = geojson.FeatureCollection(featureCollectAll)

# save as a file
with open("FEMA_Regions.geojson", 'w') as fp:
   dump(geojsonFinal, fp)

## Display Regions on a Folium/Leaflet Map
While GeoJson will automatically be displayed in GitHub, we want to display here. Since the last GeoJson set above was for all the regions, that is what will be displayed here.

In [197]:
# using folium (folium is an interface to leaflet.js) to display geojson on map
import folium

# display map by providing lat, long center point - in this case, center of USA
my_map = folium.Map(location=[42.354478452,-71.054648822],zoom_start=3)

# identify the properties elements that we want to display in the popup
folium.GeoJson(geojsonFinal,
               popup=folium.GeoJsonPopup(fields=['Region','Name','Address','City-St-Zip','States','Last-Update']),
               marker=folium.CircleMarker(radius = 5, # Radius in metres
                                          weight = 3, #outline weight
                                          fill_color = '#FFFFFF', 
                                          fill_opacity = 1)
              ).add_to(my_map)

#folium.LayerControl().add_to(my_map)
my_map